# Hate speech detection with BERT


# 0. Setup

### Installs

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.9MB/s 
     |████████████████████████████████| 901kB 36.4MB/s 
     |████████████████████████████████| 3.3MB 35.8MB/s 


### Imports

In [ ]:
# import HuggingFace models
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

# import csv to deal with .csv data files
import csv

# import pandas to just be able to visualise our data files
import pandas as pd

# classic shit
import numpy as np
import torch

# for getting right format for data
from torch.utils.data import Dataset, DataLoader

# for computing metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### Model and Tokenizer setup

In [ ]:
MODEL_NAME = "distilbert-base-uncased" # Distil BERT is a smaller model with faster execution time

# define model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, output_attentions=False, output_hidden_states=False)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side = "right")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

# 1. Data

# Twitter racism parsed dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_set = pd.read_csv("twitter_racism_parsed_dataset.csv")

In [ ]:
data_set.shape

(13471, 5)

In [ ]:
data_set.head()

,index,id,Text,Annotation,oh_label
0,5.770000e+17,5.770000e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1
1,5.410000e+17,5.410000e+17,#NULL!,none,0
2,5.680000e+17,5.680000e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1
3,5.770000e+17,5.770000e+17,"Finally I'm all caught up, and that sudden dea...",none,0
4,5.710000e+17,5.710000e+17,@carolinesinders @herecomesfran *hugs*,none,0


In [ ]:
data_set.drop(columns=['index','id', 'Annotation'], inplace=True)

In [ ]:
data_set.head()

,Text,oh_label
0,@AAlwuhaib1977 Muslim mob violence against Hin...,1
1,#NULL!,0
2,@jncatron @isra_jourisra @AMPalestine Islamoph...,1
3,"Finally I'm all caught up, and that sudden dea...",0
4,@carolinesinders @herecomesfran *hugs*,0


In [ ]:
data_set.shape

(13471, 2)

In [ ]:
# Drop rows with empty text
data_set.drop( data_set[data_set.Text.str.len() < 7].index, inplace=True)

In [ ]:
data_set.shape

(13467, 2)

In [ ]:
data_set.head()

,Text,oh_label
0,@AAlwuhaib1977 Muslim mob violence against Hin...,1
2,@jncatron @isra_jourisra @AMPalestine Islamoph...,1
3,"Finally I'm all caught up, and that sudden dea...",0
4,@carolinesinders @herecomesfran *hugs*,0
5,"Please, PLEASE start using ""is your discernmen...",0


In [ ]:
def normalise_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace(r"\#","") # replaces hashtags
    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    text = text.str.replace(r"@","")
    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.str.replace("\s{2,}", " ")
    return text

In [ ]:
data_set["Text"]=normalise_text(data_set["Text"])

In [ ]:
data_set['Text'].head()

0    aalwuhaib1977 muslim mob violence against hind...
2    jncatron isra jourisra ampalestine islamophobi...
3    finally i'm all caught up and that sudden deat...
4                  carolinesinders herecomesfran hugs 
5    please please start using "is your discernment...
Name: Text, dtype: object

In [ ]:
data_set.head()

,Text,oh_label
0,aalwuhaib1977 muslim mob violence against hind...,1
2,jncatron isra jourisra ampalestine islamophobi...,1
3,finally i'm all caught up and that sudden deat...,0
4,carolinesinders herecomesfran hugs,0
5,"please please start using ""is your discernment...",0


In [ ]:
data_set.shape

(13467, 2)

In [ ]:
data_set=  data_set.to_dict('r')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [ ]:
#data_set

In [ ]:
text_data = [row["Text"] for row in data_set]
label_data = [row["oh_label"] for row in data_set]


In [ ]:
print(len(text_data))
print(len(label_data))

13467
13467


In [ ]:
train_texts = text_data[:11500]
train_labels = label_data[:11500]

test_texts = text_data[11500:]
test_labels = label_data[11500:]

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
train_encodings[0]

Encoding(num_tokens=60, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
class TwitterRacismDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = TwitterRacismDataset(train_encodings, train_labels)
val_dataset = TwitterRacismDataset(val_encodings, val_labels)
test_dataset = TwitterRacismDataset(test_encodings, test_labels)

# 2. Training

### Set various parameters

Model hyperparameters

In [ ]:
# Hyperparameters for training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20, 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    load_best_model_at_end = True, 
    learning_rate=0.001,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    # evaluation_strategy='steps',
)

Metrics

In [ ]:
# A function computing metrics based on model output
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

### Train the model

Set model to training mode

In [ ]:
model.train()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

Define `Trainer` (training setup)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

Only train classifier layer. This code freezes all BERT layers for training.

In [ ]:
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

Finally train the model

In [ ]:
trainer.train()

Step,Training Loss
10,0.696100
20,0.533100
30,0.305800
40,0.313000
50,0.327100
60,0.290300
70,0.244600
80,0.240400
90,0.189100
100,0.337700


TrainOutput(global_step=11500, training_loss=0.1609144975050636, metrics={'train_runtime': 467.4017, 'train_samples_per_second': 24.604, 'total_flos': 4435099862400000.0, 'epoch': 20.0, 'init_mem_cpu_alloc_delta': 1955905536, 'init_mem_gpu_alloc_delta': 268953088, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 14155776, 'train_mem_gpu_alloc_delta': 7123456, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 66454528})

### Evaluate on dev set

In [ ]:
trainer.evaluate()

{'epoch': 20.0,
 'eval_accuracy': 0.9243478260869565,
 'eval_f1': 0.7528409090909091,
 'eval_loss': 0.17147615551948547,
 'eval_mem_cpu_alloc_delta': 270336,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 81737728,
 'eval_precision': 0.7817109144542773,
 'eval_recall': 0.726027397260274,
 'eval_runtime': 3.5614,
 'eval_samples_per_second': 645.808}

# 3. Testing

In this part we test our model on our test set.

Set the model to evaluate mode.

In [ ]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

Function to predict the class of an input.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_sent_pred(input_str,device=device):
    tok = tokenizer(input_str, return_tensors="pt", truncation=True, padding=True)
    tok.to(device)
    with torch.no_grad():
        pred = model(**tok)
    return pred['logits'].argmax(-1).item()

Function to compute metrics of model output for test data

In [ ]:
def compute_metrics_test(y_true,y_pred):
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    acc = accuracy_score(y_true, y_pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
}

In [ ]:
compute_metrics_test(test_labels, [get_sent_pred(sent) for sent in test_texts])

{'accuracy': 0.9242501270971022,
 'f1': 0.7537190082644627,
 'precision': 0.7261146496815286,
 'recall': 0.7835051546391752}

In [ ]:
from sklearn.metrics import matthews_corrcoef
predictions = [get_sent_pred(sent) for sent in test_texts]

In [ ]:
matthews = matthews_corrcoef(test_labels, predictions)                
matthews

0.7097534791296348